In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.nn import functional as F

In [ ]:
data = [
    {'cat1':1,'cat2':3,'target':3},
    {'cat1':1,'cat2':4,'target':1},
    {'cat1':2,'cat2':2,'target':4},
]

In [ ]:
d = pd.DataFrame(data).astype(np.float32)

In [ ]:
x_data = torch.FloatTensor(d[['cat1','cat2']].values)
y_data = F.one_hot(torch.LongTensor(d.target)).float()

In [ ]:
class LogRegression(torch.nn.Module):
    def __init__(self,num_classes=1):
        super(LogRegression, self).__init__()
        self.linear1 = torch.nn.Linear(2, 128)
        self.linear2 = torch.nn.Linear(128, 64)
        self.linear3 = torch.nn.Linear(64, num_classes)
        self.relu = torch.nn.ReLU()
        self.inference = False
    def forward(self, x):
        y_pred = self.linear3(self.linear2(self.linear1(x)))
        if self.inference:
            return torch.argmax(torch.sigmoid(y_pred),1)
        else:
            return torch.sigmoid(y_pred)
        

model = LogRegression(y_data.shape[1])

In [ ]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()

In [ ]:
model.inference=True

In [ ]:
model(x_data)

In [ ]:
torch.onnx.export(model,            
                  x_data[:1],                         
                  "t.onnx",   
                  export_params=True,        
                  do_constant_folding=True,  
                  input_names = ['input'],   
                  output_names = ['output'])